# **Ф22**: Модуль моделирования социально-экономических эффектов реализации генерального плана

Берем кварталы `контекста` и `сценария`

In [1]:
import pandas as pd
import geopandas as gpd

blocks = pd.read_pickle('./data/blocks.pickle')

Делаем предобработку: лендюзы не должны быть больше 1

In [2]:
from blocksnet.enums import LandUse

for lu in LandUse:
    blocks[lu.value] = blocks[lu.value].apply(lambda v : min(v,1))

Инициализируем `граф соседства`

In [3]:
from blocksnet.relations import generate_adjacency_graph

adjacency_graph = generate_adjacency_graph(blocks, 10)

2025-09-26 02:08:31.117 | INFO     | blocksnet.relations.adjacency.core:_generate_adjacency_nodes:9 - Generating nodes
2025-09-26 02:08:31.119 | INFO     | blocksnet.relations.adjacency.core:_generate_adjacency_edges:14 - Generating edges
2025-09-26 02:08:32.037 | SUCCESS  | blocksnet.relations.adjacency.core:generate_adjacency_graph:37 - Adjacency graph successfully generated: 2700 nodes, 8966 edges


## Параметры застройки

### FSI, MXI, GSI

Предсказываем параметры

In [4]:
from blocksnet.machine_learning.regression import DensityRegressor

dr = DensityRegressor()
density_df = dr.evaluate(blocks, adjacency_graph)
density_df.head()

2025-09-26 02:08:41.046 | INFO     | blocksnet.preprocessing.feature_engineering.core:_calculate_usual_features:35 - Calculating usual features


,fsi,gsi,mxi
0,0.062008,0.030930,0.625959
1,0.202340,0.090030,0.683971
2,0.151737,0.091165,0.075096
3,0.449142,0.139053,0.404539
4,0.079224,0.019164,0.141795


Обрабатываем результаты:
- `FSI >= 0`
- `GSI <= 1, >= 0`
- `MXI <= 1, >= 0`
- `MXI == 0` для `residential == 0`

In [5]:
density_df.loc[density_df['fsi'] < 0, 'fsi'] = 0

density_df.loc[density_df['gsi'] < 0, 'gsi'] = 0
density_df.loc[density_df['gsi'] > 1, 'gsi'] = 1

density_df.loc[density_df['mxi'] < 0, 'mxi'] = 0
density_df.loc[density_df['mxi'] > 1, 'mxi'] = 1

density_df.loc[blocks['residential'] == 0, 'mxi'] = 0

density_df.head()

,fsi,gsi,mxi
0,0.062008,0.030930,0.000000
1,0.202340,0.090030,0.000000
2,0.151737,0.091165,0.000000
3,0.449142,0.139053,0.404539
4,0.079224,0.019164,0.000000


### footprint_area, build_floor_area, living_area

In [6]:
from blocksnet.analysis.indicators import calculate_development_indicators 

density_df['site_area'] = blocks['site_area']
development_df = calculate_development_indicators(density_df)
development_df.head()

,site_area,fsi,gsi,mxi,build_floor_area,footprint_area,living_area,non_living_area
0,202399.732193,0.062008,0.030930,0.000000,12550.416350,6260.321243,0.000000,12550.416350
1,201529.493481,0.202340,0.090030,0.000000,40777.554163,18143.767788,0.000000,40777.554163
2,164663.859830,0.151737,0.091165,0.000000,24985.657278,15011.641911,0.000000,24985.657278
3,161571.832511,0.449142,0.139053,0.404539,72568.711901,22467.055572,29356.894977,43211.816923
4,16.489152,0.079224,0.019164,0.000000,1.306344,0.316006,0.000000,1.306344


### population

In [7]:
development_df['population'] = development_df['living_area'] // 20
development_df.head()

,site_area,fsi,gsi,mxi,build_floor_area,footprint_area,living_area,non_living_area,population
0,202399.732193,0.062008,0.030930,0.000000,12550.416350,6260.321243,0.000000,12550.416350,0.0
1,201529.493481,0.202340,0.090030,0.000000,40777.554163,18143.767788,0.000000,40777.554163,0.0
2,164663.859830,0.151737,0.091165,0.000000,24985.657278,15011.641911,0.000000,24985.657278,0.0
3,161571.832511,0.449142,0.139053,0.404539,72568.711901,22467.055572,29356.894977,43211.816923,1467.0
4,16.489152,0.079224,0.019164,0.000000,1.306344,0.316006,0.000000,1.306344,0.0


### Заполняем результаты предсказания в оригинальный `GeoDataFrame`

In [8]:
mask = blocks['is_project']
columns = ['build_floor_area', 'footprint_area', 'living_area', 'non_living_area', 'population']
blocks.loc[mask, columns] = development_df.loc[mask, columns]

## Предсказание СЭР

Немного преобразуем данные

In [9]:
from blocksnet.enums import LandUse

# переводим доли ЛУ в площади ЛУ
for lu in LandUse:
    blocks[lu.value] = blocks[lu.value] * blocks['site_area'] 

# суммируем всё строки
data = [blocks.drop(columns=['land_use', 'geometry']).sum().to_dict()]
input = pd.DataFrame(data)

# добавляем координаты центра
input['latitude'] = blocks.geometry.union_all().centroid.x
input['longitude'] = blocks.geometry.union_all().centroid.y

#переименуем count_buildings
input['buildings_count'] = input['count_buildings']

Предсказываем

In [10]:
from blocksnet.machine_learning.regression import SocialRegressor

sr = SocialRegressor()
y_pred, pi_lower, pi_upper = sr.evaluate(input)

## Результат

In [11]:
iloc = 0
result_data = {
    'pred': y_pred.apply(round).astype(int).iloc[iloc].to_dict(),
    'lower': pi_lower.iloc[iloc].to_dict(),
    'upper': pi_upper.iloc[iloc].to_dict(),
}
result_df = pd.DataFrame.from_dict(result_data)
result_df['is_interval'] = (result_df['pred'] <= result_df['upper']) & (result_df['pred'] >= result_df['lower'])

In [12]:
result_df

,pred,lower,upper,is_interval
nursing_home_count,1,0.0,0.400027,False
hotel_count,45,0.0,66.670968,True
theatre_count,5,0.0,5.484691,True
cinema_count,1,0.0,1.590170,True
secondary_vocational_education_institutions_count,8,0.0,6.643087,False
university_count,2,0.0,8.539938,True
stadium_count,2,0.0,6.275497,True
emergency_medical_service_stations_count,2,0.0,1.253555,False
kindergarten_count,23,0.0,23.841917,True
hostel_count,35,0.0,18.957263,False
